In [94]:
from random import sample
import numpy as np
import pandas as pd
class Die():
    
    def __init__(self, sides):
        if type(sides) != np.ndarray:
            raise TypeError("The input must be a numpy array")
        if len(sides) != len(set(sides)):
            raise ValueError("Array values are not unique")
        if not np.issubdtype(sides.dtype, np.number) and not np.issubdtype(sides.dtype, np.str_):
            raise TypeError("The input array must have a data type of strings or numbers")
        self.sides = sides
        self.weights = np.ones(len(self.sides))
        self._die = pd.DataFrame({'side': self.sides, 'weight': self.weights/len(sides)})
    
    def side_weight_change(self, side, weight):
        if side not in self._die['side'].values:
            raise IndexError("The side does not exist")
        if weight < 0:
            raise ValueError("The weight must be greater than or equal to zero")
        if not np.issubdtype(type(weight), np.number):
            raise TypeError("The weight must be a number")
        self._die.loc[self._die['side'] == side, 'weight'] = weight
        self.weights = self._die['weight'].values
    
    def roll(self, nrolls):
        self.nrolls = 1
        total_weight = sum(self.weights)
        normalized_weights = [w / total_weight for w in self.weights]
        outcomes = np.random.choice(self._die['side'].values, size=nrolls, replace=True, p=normalized_weights)
        return list(outcomes)
    
    def current_state(self):
        return self._die


In [95]:
sides = np.array([1,2,3,4,5,6])
game = Die(sides)
game.side_weight_change(1, 0.5)
game._die
#game = Die(np.array([1,2,3,4,5,6]))
#print(game.current_state())
#game.nrolls = 15
#print(game.roll(5))

,side,weight
0,1,0.500000
1,2,0.166667
2,3,0.166667
3,4,0.166667
4,5,0.166667
5,6,0.166667


In [96]:
#unit test for first error
#game = Die([1,2,3,4,5,6])

In [97]:
class Game():
    """
    A class representing a game with multiple dice.

    Parameters:
    dice (list): A list of dice objects representing the different dice in the game.

    Attributes:
    dice (list): A list of dice objects representing the different dice in the game.
    dice_df (pandas.DataFrame): A DataFrame containing information about the dice.
    _nrolls (int): The number of rolls for the game.
    _gamedf (pandas.DataFrame): A DataFrame containing the game results.

    Methods:
    play(die_number, nrolls): Plays the game with the specified die number and number of rolls.
    results(form='wide'): Returns the game results in the specified form.

    """

    def __init__(self, dice):
        """
        Initializes a Game object.

        Parameters:
        dice (list): A list of dice objects representing the different dice in the game.

        Raises:
        TypeError: If the input is not a list.

        """
        if type(dice) != list:
            raise TypeError("The input must be a list")
        self.dice = dice
        self.dice_df = pd.DataFrame({
            'dice_num': range(1, len(dice) + 1),
            'die': dice
        })
        self._nrolls = None
        self._gamedf = pd.DataFrame()
    
    def play(self, die_number, nrolls):
        """
        Plays the game with the specified die number and number of rolls.

        Parameters:
        die_number (int): The number of the die to play with.
        nrolls (int): The number of rolls for the game.

        Raises:
        ValueError: If the number of rolls is incompatible with previous rolls.

        Returns:
        pandas.DataFrame: The game results.

        """
        if self._nrolls is None:
            self._nrolls = nrolls
        elif self._nrolls != nrolls:
            raise ValueError(f"Incompatible number of rolls: {nrolls} provided, but {self._nrolls} expected")
        
        spec_die = self.dice[die_number - 1]  # Access the specific die by index (adjusting for 1-based indexing)
        rolls = spec_die.roll(nrolls)
        
        # Create a DataFrame to store the results in wide format
        roll_df = pd.DataFrame({
            'roll_number': range(1, nrolls + 1),
            f'die_{die_number}': rolls
        }).set_index('roll_number')
        
        # Combine with existing _gamedf
        if self._gamedf.empty:
            self._gamedf = roll_df
        else:
            self._gamedf = self._gamedf.join(roll_df, how='outer')
        
        return self._gamedf
        
    def results(self, form='wide'):
        """
        Returns the game results in the specified form.

        Parameters:
        form (str): The form of the game results. Defaults to 'wide'.

        Raises:
        ValueError: If the form is not a valid option.

        Returns:
        pandas.DataFrame: The game results.

        """
        self.form = form
        if self.form == 'wide':
            return self._gamedf
        elif self.form == 'narrow':
            return self._gamedf.melt(ignore_index=False)
        else:
            raise ValueError("Invalid option for form")


In [98]:
my_list = [Die(np.array([1,2,3,4,5,6])), Die(np.array([1,2,3,4,5,6]))]
game = Game(my_list)
game.play(1, 5)
game.play(2, 5)
#game.results().shape[1]
game._gamedf.shape[1]
#game.results('narrow')

2

## The Analyzer class

### General Definition

An Analyzer object takes the results of a single game and computes
various descriptive statistical properties about it.

### Specific Methods and Attributes

**An initializer**.

-   Takes a game object as its input parameter. Throws a `ValueError` if
    the passed value is not a Game object.

**A jackpot method.**

-   A jackpot is a result in which all faces are the same, e.g. all ones
    for a six-sided die.

-   Computes how many times the game resulted in a jackpot.

-   Returns an integer for the number of jackpots.

**A face counts per roll method.**

-   Computes how many times a given face is rolled in each event.

    -   For example, if a roll of five dice has all sixes, then the
        counts for this roll would be $5$ for the face value ‘6’ and $0$
        for the other faces.

-   Returns a data frame of results.

-   The data frame has an index of the roll number, face values as
    columns, and count values in the cells (i.e. it is in wide format)..

**A combo count method.**

-   Computes the distinct combinations of faces rolled, along with their
    counts.

-   Combinations are order-independent and may contain repetitions.

-   Returns a data frame of results.

-   The data frame should have a MultiIndex of distinct combinations
    and a column for the associated counts.

**An permutation count method.**

-   Computes the distinct permutations of faces rolled, along with their
    counts.

-   Permutations are order-dependent and may contain repetitions.

-   Returns a data frame of results.

-   The data frame should have a MultiIndex of distinct permutations
    and a column for the associated counts.

In [99]:
class Analyzer():
    
    def __init__(self, game):
        if type(game) != Game:
            raise TypeError("The input must be a Game object")
        self.game = game
        self._gamedf = game._gamedf
        self.nrolls = len(game._gamedf) 
        self.jackpot_counter = 0
        self.die_sides = self.game.dice[0].sides

    def jackpot(self):
        for index, row in self._gamedf.iterrows():
            if len(set(row)) == 1:  # Check if all non-NaN values in the row are the same
                self.jackpot_counter += 1
        return self.jackpot_counter
    
    def face_count(self):
        counts = []
        for _, row in self.game._gamedf.iterrows():
            count = row.value_counts().reindex(self.die_sides, fill_value=0)
            counts.append(count)
        counts_df = pd.DataFrame(counts, index=self.game._gamedf.index).reset_index(drop=True)
        return counts_df
        
    def combo_count(self):
        #computes distinct combinations of faces rolled
        combos = []
        for _, row in self.game._gamedf.iterrows():
            combo = tuple(sorted(row.dropna()))
            combos.append(combo)
        combo_counts = pd.Series(combos).value_counts().reset_index()
        combo_counts.columns = ['combination', 'count']
        combo_counts.set_index('combination', inplace=True)
        return combo_counts
    
    def perm_count(self):
        #computes distinct permutations of faces rolled
        perms = []
        for _, row in self.game._gamedf.iterrows():
            perm = tuple(row.dropna())
            perms.append(perm)
        perm_counts = pd.Series(perms).value_counts().reset_index()
        perm_counts.columns = ['permutation', 'count']
        perm_counts.set_index('permutation', inplace=True)
        return perm_counts
        

In [153]:
Die1 = Die(np.array([1,2,3,4,5,6]))
#Die1.side_weight_change(1, 0.3)
Die2 = Die(np.array([1,2,3,4,5,6]))
#Die2.side_weight_change(1, 0.3)
game = Game([Die1, Die2])
game.play(1, 50)
game.play(2, 50)
analyzer = Analyzer(game)
var1 = analyzer.perm_count()
var1 #dataframe
sum(var1['count'])
game._nrolls

#len(var1.index)


#if true, length of faces.columns is equal to the number of sides of the die
#len(faces.columns)
#len(Die1.sides)
#len(faces.columns)

#analyzer.combo_count()
#analyzer.perm_count()

50